Importing necessary libraries and supporting files

In [1]:
import pandas as pd
import numpy as np
import xlwt
from xlwt import Workbook
from datetime import date
import math
import xlsxwriter
from pandas import read_excel

Available = read_excel('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Available.xlsx', sheet_name = 'Report1')
Down = read_excel('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Down.xlsx', sheet_name = 'Report1')
Leased = read_excel('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Leased.xlsx', sheet_name = 'Report1')

Residents = read_excel('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Residents.xlsx', sheet_name = 'Report1')
Yardi = read_excel('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Yardi.xlsx', sheet_name = 'Report1')
Yardi = Yardi.sort_values(by = ['Property', 'Unit'], ignore_index=True)

Light5 = read_excel('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Light5.xlsx', sheet_name = 'Light 0.5 Tracker')
Light5.columns = Light5.iloc[0]
Light5.drop(Light5.index[0], inplace = True)

PC = read_excel('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/PC.xlsx', sheet_name = 'PC Tracker')
PC.columns = PC.iloc[0]
PC.drop(PC.index[0], inplace = True)

Access = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Access.csv', error_bad_lines=False)
Procore = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/Procore.csv')
PropertyLookUpTable = pd.read_csv ('C:/Users/mitch_ue52/Documents/GrandInterface/Excel/PropLookUp.csv')

This function creates a column titled 'Combo' which contains <Property Number>_<Unit Number>
it also eliminates any leading 0's any '.0' which is sometimes found in the Property Number

In [2]:
def makeComboUnits(db):
    unitNum = db['Unit'].astype(str)
    ind = 0
    for val in unitNum:
        while True:
            if (val[0] == '0'):
                val = val[1:]
            else:
                unitNum[ind] = val
                ind = ind + 1
                break
                
    db['Combo'] = db['Property'].astype(str) + '_' +unitNum
    db['Combo'] = db['Combo'].str.replace('.0', '', regex=False)
    db['Combo'] = db['Combo'].str.strip()

In [3]:
def makeComboUnitsYardiFunction(db):
    db['Property'] = db['Property'].str.strip()
    db['Unit'] = db['Unit'].str.strip()
    unitNum = db['Unit'].astype(str)
    ind = 0
    for val in unitNum:
        while True:
            if (val[0] == '0'):
                val = val[1:]
            else:
                unitNum[ind+1] = val
                ind = ind + 1
                break
                
    db['Combo'] = db['Property'].astype(str) + '_' +unitNum
    db['Combo'] = db['Combo'].str.replace('.0', '', regex=False)
    db['Combo'] = db['Combo'].str.strip()
    db['Actual \nCompletion Date'] =  db['Actual \nCompletion Date'].apply(lambda x: str(x)[:10])

Calling the function with the supporting data

In [4]:
Access = Access.applymap(str)

Residents ['Move In'] = Residents['Move In'].apply(lambda x: str(x)[:10])
Residents ['Move Out'] = Residents['Move Out'].apply(lambda x: str(x)[:10])


CurrentResidents = Residents.loc[(Residents['Status'] == 'Current') | (Residents['Status'] == 'Future')]
PastResidents = Residents.loc[Residents['Status'] == 'Past']
CurrentResidents = pd.DataFrame.reset_index(CurrentResidents)
PastResidents = pd.DataFrame.reset_index(PastResidents)
PastResidents = PastResidents.sort_values(by = 'Move Out', ignore_index=True, ascending = False)


makeComboUnits(Yardi)
makeComboUnits(Available)
makeComboUnits(Down)
makeComboUnits(Leased)
makeComboUnits(CurrentResidents)
makeComboUnits(PastResidents)

makeComboUnitsYardiFunction(Light5)
makeComboUnitsYardiFunction(PC)

This is where the weird logic is located.  Instead of trying to determine the unit numbers from each Procore project, it is easier to simply break up each word in the Procore Project and create a combo unit with it.  For example if the job is 
'225 Mallorca 101, +205, (301), -307'
this will create combo string of 
['4301_225', '4301_Mallorca', '4301_101', '4301_+205', 4301_(301)', '4301_-307']
We will store this information into a dataframe called 'Procore_Units'

We can then see if any of our Down units are found in this list which is the end goal.

Some small workarounds we need to do with this approach:
    Ignore the first word in the Procore Project string, if there exists a 225 Mallorca unit 225 this will create issues.
    Remove any '+' ',' or ' ', that might create some bugs down the line.

In [5]:
Procore['Property Number'] = Procore['Project Number'].astype(str)

Procore_Units = pd.DataFrame(columns = ['Combo', 'Stage'])
for proj in Procore['Name']:
    terms = proj.split()
    index = 1
    while True:
        
        unitNum = terms[index]
        unitNum = str(unitNum)
        while True:
            if (unitNum != ''):
                if (unitNum[0] == '0'):
                    unitNum = unitNum[-1*(len(unitNum))+1:]
                else:
                    break
            else:
                break

        projNum = Procore.loc[Procore.Name == proj, 'Project Number'].values[0]
        comboStr = str(projNum) + '_'+ str(unitNum)
       
        deletedChar = [',', ' ', '+']
       
        for char in deletedChar:
            comboStr = comboStr.replace(char,'')
            
        stageStr = Procore.loc[Procore.Name == proj, 'Stage']
        Procore_Units = Procore_Units.append({'Combo': comboStr, 'Stage': stageStr }, ignore_index=True)        
        index = index + 1
        
        if (index == len(terms)):
            break
        
Procore_Units['Combo'] = Procore_Units['Combo'].str.replace('.0', '', regex=False)

In [6]:
Yardi['Status'] = np.nan
Yardi['Procore Status'] = np.nan
Yardi['Light5 Status'] = np.nan
Yardi['PC Status'] = np.nan
Yardi['Move In'] = np.nan
Yardi['Move Out'] = np.nan
Yardi['Resident Name'] = np.nan
Yardi['Resident Number'] = np.nan

for unit in Yardi['Combo']:
    
    if (unit in Procore_Units['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Procore Status'] =  Procore_Units.loc[Procore_Units.Combo==unit,'Stage'].values[0].values[0]
    else:
        Yardi.loc[Yardi.Combo == unit, 'Procore Status'] = 'N/A'

    
    if (unit in Light5['Combo'].values):
        if (Light5.loc[Light5.Combo == unit, 'Job Status'].values[0] == 'Completed'):
            Yardi.loc[Yardi.Combo == unit, 'Light5 Status'] = Light5.loc[Light5.Combo == unit, 'Job Status'].values[0] + ' - ' + Light5.loc[Light5.Combo == unit, 'Actual \nCompletion Date'].values[0]
        else:
            Yardi.loc[Yardi.Combo == unit, 'Light5 Status'] = Light5.loc[Light5.Combo == unit, 'Job Status'].values[0]
    else:
        Yardi.loc[Yardi.Combo == unit, 'Light5 Status'] = 'N/A'

                
    if (unit in PC['Combo'].values):
        PCHold = PC[PC['Combo']==unit]
        PCHold = PCHold.sort_values(by = 'Job Code', ignore_index=True, ascending=False)
        
        if (PCHold['Job Status'][0] == 'Completed'):
            Yardi.loc[Yardi.Combo == unit, 'PC Status'] = PCHold['Job Status'][0] + ' - ' + PCHold['Actual \nCompletion Date'].values[0]
        else:
            Yardi.loc[Yardi.Combo == unit, 'PC Status'] = PCHold['Job Status'][0]
    else:
        Yardi.loc[Yardi.Combo == unit, 'PC Status'] = 'N/A'

    
    if (unit in Available['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Status'] = 'Available'
        
    elif (unit in Down['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Status'] = 'Down'
        
    elif (unit in Leased['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Status'] = 'Leased'
      
    else:
        print(unit)        
        
    if (unit in CurrentResidents['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Move In'] =  CurrentResidents.loc[CurrentResidents.Combo==unit,'Move In'].values[0]
        Yardi.loc[Yardi.Combo == unit, 'Resident Name'] =  CurrentResidents.loc[CurrentResidents.Combo==unit,'Name'].values[0]
        Yardi.loc[Yardi.Combo == unit, 'Resident Number'] =  CurrentResidents.loc[CurrentResidents.Combo==unit,'Phone #s'].values[0]
        Yardi.loc[Yardi.Combo == unit, 'Move Out'] = 'N/A'
        
    elif (unit in PastResidents['Combo'].values):
        Yardi.loc[Yardi.Combo == unit, 'Move Out'] =  PastResidents['Move Out'][PastResidents.loc[PastResidents.Combo == unit].first_valid_index()]
        

    else:
        Yardi.loc[Yardi.Combo == unit, 'Resident Name'] = 'N/A'
        Yardi.loc[Yardi.Combo == unit, 'Resident Number'] = 'N/A'

In [7]:
class FitSheetWrapper(object):
    def __init__(self, sheet):
        self.sheet = sheet
        self.widths = dict()

    def write(self, r, c, label='', *args, **kwargs):
        self.sheet.write(r, c, label, *args, **kwargs)
        width = 5000
        self.sheet.col(c).width = width
#         if width > self.widths.get(c, 0):
#             self.widths[c] = width
#             self.sheet.col(c).width = width

    def __getattr__(self, attr):
        return getattr(self.sheet, attr)

Here we use the PropertyLookUpTable to get the property name for each project in Yardi.  This will help us later when creating the Grand Interface output file.  We will also reorganize the Yardi dataframe so it's easier to print later

In [8]:
PropertyLookUpTable['YARDI Code'].apply(lambda x: str(x))
propCodesDict = dict(zip(PropertyLookUpTable['YARDI Code'], PropertyLookUpTable['Cleaned Property']))
Yardi['Property Name'] = list(map(propCodesDict.get, Yardi['Property']))
Yardi = Yardi[['Property Name', 'Property', 'Unit', 'Status', 'Combo', 'Resident Name', 'Resident Number', 'Move In', 'Move Out', 'Unit Type', 'Procore Status', 'Light5 Status', 'PC Status']]

accessPropCodes = Access['Property Code'].astype(int)

Access['Property Name'] = list(map(propCodesDict.get, accessPropCodes))
Access = Access[['Property Name', 'Pointer', 'Property Code', 'Intercom Code', 'Front Entrance Code',
       'LockBox Code', 'Boiler Room Access', 'Master Code',
       'Lockbox Location 1', 'Lockbox Code 1', 'Lockbox Location 2',
       'Lockbox Code 2', 'Lockbox Location 3', 'Lockbox Code 3', 'Notes']]

Access = Access.sort_values(by = ['Property Name'], ignore_index=True)

This is where all of the printing to Excel happens now.  We have the status values and everything needed.  We pull values from the Property Look Up file to determine the PM, PA, and RM and then plot everything.

Access Sheet

In [9]:
for column in columnTitles:
    print("column: " + column)
    print(Access[column])

NameError: name 'columnTitles' is not defined

In [ ]:
today = date.today()
wb = Workbook()
newSheet = FitSheetWrapper(wb.add_sheet('Access Info'))
columnTitles = list(Access.columns.values)
columnTitles.remove('Pointer')
columnTitles.remove('Property Code')


row = 0
col = 0

for column in columnTitles:
    row = 0
    newSheet.write(row,col,column)
    for val in Access[column]:
        row = row + 1
        newSheet.write(row,col,val)
    col = col + 1





fileName = ('Access ' + str(today) + '.xls')
wb.save(fileName)

ListView

In [ ]:
today = date.today()
fileName = ('LI ' + str(today) + '.xls')
Yardi.to_excel(fileName, index = False, freeze_panes=(1,1))

Building View

In [ ]:
wb = Workbook()
Yardi = Yardi.sort_values(by = ['Property Name', 'Unit'], ignore_index=True)

index = 0
length = len(Yardi) - 1

while True:
    if (index == length):
        break
    prop = str(Yardi['Property'][index])
    propIndex = list(PropertyLookUpTable['YARDI Code']).index(int(Yardi['Property'][index]))    
    propCombo = str(Yardi['Property'][index]) + ' - ' + str(Yardi['Property Name'][index])    
    propMgr = 'PM: ' + PropertyLookUpTable['Prop Mgr'][propIndex]
    propAdmin = 'PA: ' + PropertyLookUpTable['Prop Admin'][propIndex]
    
    if (type(PropertyLookUpTable['Resident Manager'][propIndex]) == str):
        propRM = 'RM: ' + PropertyLookUpTable['Resident Manager'][propIndex]    
    else:
        propRM = 'No RM'
        
    accessIndex = list(Access['Property Code']).index(str(Yardi['Property'][index]))    
    intercomCode = 'Intercom: ' + Access['Intercom Code'][accessIndex]
    frontEntranceCode = 'Front Entrance: ' + Access['Front Entrance Code'][accessIndex]
    lockboxCode = 'Lockbox: ' + Access['LockBox Code'][accessIndex]
    boilerCode = 'Boiler: ' + Access['Boiler Room Access'][accessIndex]
    masterCode = 'Master: ' + Access['Master Code'][accessIndex]
    lockbox1 = 'Lockbox 1: ' + Access['Lockbox Location 1'][accessIndex] + ' - ' + Access['Lockbox Code 1'][accessIndex]
    lockbox2 = 'Lockbox 2: ' + Access['Lockbox Location 2'][accessIndex] + ' - ' + Access['Lockbox Code 2'][accessIndex]
    lockbox3 = 'Lockbox 3: ' + Access['Lockbox Location 3'][accessIndex] + ' - ' + Access['Lockbox Code 3'][accessIndex]
    notes = 'Notes: ' + Access['Notes'][accessIndex]
       
    newSheet = FitSheetWrapper(wb.add_sheet(str(Yardi['Property Name'][index])))
  
    row = 0
    col = 0
    
    propInfo = [propCombo, propMgr, propAdmin, propRM]
    accessInfo = [intercomCode, frontEntranceCode, lockboxCode, boilerCode, masterCode, lockbox1, lockbox2, lockbox3, notes]
    
    for info in propInfo:
        newSheet.write(row,col,info)
        col = col + 1

        
    row = 1
    col = 0   

    for info in accessInfo:
        newSheet.write(row,col,info)
        col = col + 1
    
    row = 3
    col = 0
    
    
    columnTitles = list(Yardi.columns.values)
    columnTitles.remove('Combo')
    columnTitles.remove('Property')

    
    for columnTitle in columnTitles:
        newSheet.write(row, col, columnTitle)
        col = col + 1
    
    
    row = 4
    col = 0
    while True:
        col = 0
        for Column in columnTitles:
            val = str(Yardi[Column][index])            
            newSheet.write(row, col, val)
            col = col + 1        
        
        index = index + 1
        row = row + 1
        
        if(index == length):
            break
        if(prop != str(Yardi['Property'][index])):
            break

In [ ]:
today = date.today()
fileName = ('GI ' + str(today) + '.xls')

wb.save(fileName)